
## 1 Importing the necessary libraries

Run the cell below to import the necessary libraries for this assignment.

In [ ]:
from utils import (
    retrieve, 
    pprint, 
    generate_with_single_input, 
    read_dataframe, 
    display_widget
)
import unittests

## 2 - Loading the dataset

In [ ]:
NEWS_DATA = read_dataframe("news_data_dedup.csv")

Let's check the data structure.

In [ ]:
pprint(NEWS_DATA[9:12])

In [ ]:
def query_news(indices):
    """
    Retrieves elements from a dataset based on specified indices.

    Parameters:
    indices (list of int): A list containing the indices of the desired elements in the dataset.
    dataset (list or sequence): The dataset from which elements are to be retrieved. It should support indexing.

    Returns:
    list: A list of elements from the dataset corresponding to the indices provided in list_of_indices.
    """
     
    output = [NEWS_DATA[index] for index in indices]

    return output

In [ ]:
# Fetching some indices
indices = [3, 6, 9]
pprint(query_news(indices))

In [ ]:
# Let's test the retrieve function!
indices = retrieve("Regulations about Autopilot", top_k = 1)
print(indices)

In [ ]:
# Now let's query the corresponding news_
retrieved_documents = query_news(indices)
pprint(retrieved_documents)

In [ ]:
# GRADED CELL 

def get_relevant_data(query: str, top_k: int = 5) -> list[dict]:
    """
    Retrieve and return the top relevant data items based on a given query.

    This function performs the following steps:
    1. Retrieves the indices of the top 'k' relevant items from a dataset based on the provided `query`.
    2. Fetches the corresponding data for these indices from the dataset.

    Parameters:
    - query (str): The search query string used to find relevant items.
    - top_k (int, optional): The number of top items to retrieve. Default is 5.

    Returns:
    - list[dict]: A list of dictionaries containing the data associated 
      with the top relevant items.

    """
    ### START CODE HERE ###

    # Retrieve the indices of the top_k relevant items given the query
    relevant_indices = retrieve(query, top_k)

    # Obtain the data related to the items using the indices from the previous step
    relevant_data = query_news(relevant_indices)

    ### END CODE HERE
    
    return relevant_data

In [ ]:
query = "Greatest storms in the US"
relevant_data = get_relevant_data(query, top_k = 1)
pprint(relevant_data)

In [ ]:
# Run this cell to perform several tests on your function. If you receive "All test passed!" it means that your solution will likely pass the autograder too.
unittests.test_get_relevant_data(get_relevant_data)

## Formatting the relevant Data

 It’s recommended to use double quotes for strings and single quotes for dictionary keys (for example, data['title']). Here’s one way you could format it:

f"""
Title: {news_1_title}, Description: {news_1_description}, Published at: {news_1_published_date}\nURL: {news_1_URL}
Title: {news_2_title}, Description: {news_2_description}, Published at: {news_2_published_date}\nURL: {news_2_URL}
...
Title: {news_k_title}, Description: {news_k_description}, Published at: {news_k_published_date}\nURL: {news_k_URL}
"""

In [ ]:
# GRADED CELL

def format_relevant_data(relevant_data):
    """
    Retrieves the top_k most relevant documents based on a given query and constructs an augmented prompt for a RAG system.

    Parameters:
    relevant_data (list): A list with relevant data.

    Returns:
    str: An augmented prompt with the top_k relevant documents, formatted for use in a Retrieval-Augmented Generation (RAG) system."
    """

    ### START CODE HERE ###

    # Create a list so store the formatted documents
    formatted_documents = []
    
    # Iterates over each relevant document.
    for document in relevant_data:

        # Formats each document into a structured layout string. Remember that each document is in one different line. So you should add a new line character after each document added.
        formatted_document = f"Title: {document['title']}, Description: {document['description']}, Published at: {document['published_at']}\nURL: {document['url']}"
        
        # Append the formatted document string to the formatted_documents list
        formatted_documents.append(formatted_document)
    
    ### END CODE HERE ###
    
    # Returns the final augmented prompt string.

    return "\n".join(formatted_documents)

In [ ]:
example_data = NEWS_DATA[4:8]
print(format_relevant_data(example_data))

In [ ]:
# Test your function!
unittests.test_format_relevant_data(format_relevant_data)

## 3.5 Generate the final prompt

The next function is given to you. It will generate the final prompt, integrating it with the query. Feel free to change the prompt and experiment how different prompts impact the final result!

In [ ]:
# EDITABLE CELL

def generate_final_prompt(query, top_k=5, use_rag=True, prompt=None):
    """
    Generates a final prompt based on a user query, optionally incorporating relevant data using retrieval-augmented generation (RAG).

    Args:
        query (str): The user query for which the prompt is to be generated.
        top_k (int, optional): The number of top relevant data pieces to retrieve and incorporate. Default is 5.
        use_rag (bool, optional): A flag indicating whether to use retrieval-augmented generation (RAG)
                                  by including relevant data in the prompt. Default is True.
        prompt (str, optional): A template string for the prompt. It can contain placeholders {query} and {documents}
                                for formatting with the query and formatted relevant data, respectively.

    Returns:
        str: The generated prompt, either consisting solely of the query or expanded with relevant data
             formatted for additional context.
    """
    # If RAG is not being used, format the prompt with just the query or return the query directly
    if not use_rag:
        return query

    # Retrieve the top_k relevant data pieces based on the query
    relevant_data = get_relevant_data(query, top_k=top_k)

    # Format the retrieved relevant data
    retrieve_data_formatted = format_relevant_data(relevant_data)

    # If no custom prompt is provided, use the default prompt template
    if prompt is None:
        prompt = (
            f"Answer the user query below. There will be provided additional information for you to compose your answer. "
            f"The relevant information provided is from 2024 and it should be added as your overall knowledge to answer the query, "
            f"you should not rely only on this information to answer the query, but add it to your overall knowledge."
            f"Query: {query}\n"
            f"2024 News: {retrieve_data_formatted}"
        )
    else:
        # If a custom prompt is provided, format it with the query and formatted relevant data
        prompt = prompt.format(query=query, documents=retrieve_data_formatted)

    return prompt

In [ ]:
print(generate_final_prompt("Tell me about the US GDP in the past 3 years."))

In [ ]:
print(generate_final_prompt("Tell me about paris"))

## 3.6 LLM call
Now let's integrate the function above to feed an LLM. Its parameters are:

query: the query to be passed to the LLM.
use_rag: a boolean telling whether using RAG or not. This parameter will help you compare queries using a RAG system and not using it.
model: the model to be used. You might change it, but the standard is the Llama 3 Billion parameter.

In [ ]:
def llm_call(query, top_k = 5, use_rag = True, prompt = None):
    """
    Calls the LLM to generate a response based on a query, optionally using retrieval-augmented generation.

    Args:
        query (str): The user query that will be processed by the language model.
        use_rag (bool, optional): A flag that indicates whether to use retrieval-augmented generation by 
                                  incorporating relevant documents into the prompt. Default is True.

    Returns:
        str: The content of the response generated by the language model.
    """
    

    # Get the prompt with the query + relevant documents
    prompt = generate_final_prompt(query, top_k, use_rag, prompt)

    # Call the LLM
    generated_response = generate_with_single_input(prompt)

    # Get the content
    generated_message = generated_response['content']
    
    return generated_message

In [ ]:
query = "Tell me about the US GDP in the past 3 years."

In [ ]:
print(llm_call(query, use_rag = True))

In [ ]:
print(llm_call(query, use_rag = False))

## 4 - Experimenting with your RAG System

Now you can experiment with your own queries to see the system in action! You can write any query, and it will display answers both with and without RAG. Keep in mind that the dataset you're working with is related to news data from 2024, so not all queries will be effective in demonstrating the framework. Some example queries you might try include:

What were the most important events of the past year?
How is global warming progressing in 2024?
Tell me about the most recent advances in AI.
Give me the most important facts from past year.
You can also specify a layout for the augmented prompt that includes placeholders for {query} and {documents} to indicate where they should be inserted within your prompt structure. For example:

This is the query: {query}
These are the documents: {documents}

In [ ]:
display_widget(llm_call)